# Setup & Utils

In [ ]:
import requests

In [ ]:
import re

def extract_model_response(text: str) -> str:
    """
    Extracts the content between <think> and the final <|eot_id|> marker.

    Parameters:
        text (str): The full text containing <think> and <|eot_id|> markers.

    Returns:
        str: The extracted content between <think> and <|eot_id|>, or an empty string if not found.
    """
    match = re.search(r"<think>(.*?)<\|end_of_text\|>", text, re.DOTALL)
    return match.group(1).strip() if match else ""


**Please enter the API key that was provided to you**

In [ ]:
API_KEY = ""

# Use Case - Incident Summarization

**Use Case Description** – Critical cybersecurity incidents often result in lengthy, complex documentation that is difficult to digest, slowing triage, handoffs, and leadership visibility. The goal is to automatically condense these incident reports into concise summaries tailored to analysts or executives — enabling faster response, clearer communication, and more effective decision-making.

## Incident Summarization Utils

In [ ]:
def create_incident_summary_prompt(incident_document: str, summary_type: str = "executive") -> str:
    """Generate a prompt to summarize long-form cybersecurity incident documentation."""
    if summary_type == "executive":
        instructions = (
            "You are a SOC assistant summarizing a lengthy incident report.\n"
            "Your task is to provide a clear, non-technical summary for security leadership.\n"
            "- Focus on what happened, when it happened, and why it matters\n"
            "- Keep it under 5 sentences\n"
            "- Use clear, plain language"
        )
    elif summary_type == "tactical":
        instructions = (
            "You are a SOC assistant extracting key technical findings from a lengthy incident report.\n"
            "Your task is to provide a concise bullet-point summary for L2/L3 analysts.\n"
            "- List key events and techniques\n"
            "- Use technical language where appropriate"
        )
    else:
        raise ValueError("Invalid summary_type. Use 'executive' or 'tactical'.")

    return f"""
{instructions}

## INCIDENT DOCUMENTATION
{incident_document}

Respond with a clear summary.
"""

In [ ]:
def run_incident_summary_analysis(incident_document: str, summary_type: str = "executive") -> str:
    """Send incident document to AI model and return a summary."""
    prompt = create_incident_summary_prompt(incident_document, summary_type)

    resp = requests.post(
        "https://model-dq4r501w.api.baseten.co/development/predict",
        headers={"Authorization": f"Api-Key {API_KEY}"},
        json={"prompt": prompt},
    )

    return extract_model_response(resp.text)

## Example Usage

In [ ]:
long_incident_doc = """
Incident Report: IR-2025-0173

Executive Summary:
On April 4, 2025, the security operations center (SOC) was alerted to anomalous behavior originating from an engineering jump box (ENG-JB01). The incident spanned approximately 18 hours and resulted in the compromise of multiple internal systems, theft of sensitive source code repositories, and signs of attempted exfiltration to a known threat infrastructure host associated with APT29. This report outlines the timeline, impact, attacker techniques, and remediation actions taken.

Timeline of Events:
- 2025-04-04 01:14 UTC: Alert triggered for unusual lateral movement attempt originating from ENG-JB01. Logged by Cisco XDR as “Possible Pass-the-Hash movement attempt.”
- 2025-04-04 01:18 UTC: Multiple failed Kerberos authentication attempts on host HR-LAPTOP-5 from ENG-JB01.
- 2025-04-04 01:23 UTC: Successful logon to FIN-SQL-02 using a privileged service account (svc-backup-prod) outside of usual login times.
- 2025-04-04 01:25 UTC: PowerShell execution detected from FIN-SQL-02 running a Base64-encoded command fetching content from hxxp://185.44.76.100/payload.ps1.
- 2025-04-04 01:30 UTC: Scheduled task created named “WindowsHealth” on FIN-SQL-02 for persistence.
- 2025-04-04 01:33 UTC: Internal network scanning activity detected from FIN-SQL-02 targeting subnet 10.0.42.0/24.
- 2025-04-04 02:01 UTC: SMB and WMI-based authentication attempts observed toward code repository server (ENG-GIT-SRV).
- 2025-04-04 02:11 UTC: Access to sensitive Git repositories verified using server audit logs.
- 2025-04-04 02:15–04:30 UTC: Multiple 50–150MB file transfers from ENG-GIT-SRV to ENG-JB01. Total volume approx. 3.1GB.
- 2025-04-04 04:45 UTC: Outbound connection to 185.44.76.100 established from ENG-JB01 on port 443, observed via egress firewall logs.
- 2025-04-04 04:52 UTC: Data transfer spike of ~3.2GB to external host.
- 2025-04-04 05:02 UTC: DLP system flagged unauthorized data exfiltration attempt. Incident escalated to IR team.
- 2025-04-04 05:13 UTC: Isolation actions initiated on ENG-JB01, FIN-SQL-02, and ENG-GIT-SRV.
- 2025-04-04 05:55 UTC: Memory capture and forensic triage initiated on all impacted systems.

Tactics and Techniques:
- Initial Access: Credential stuffing or lateral movement via Jump Box (ENG-JB01)
- Credential Access: Possible Pass-the-Hash (T1550.002), credential dumping via LSASS access
- Execution: Obfuscated PowerShell scripts (T1059.001)
- Persistence: Scheduled Task creation (T1053.005)
- Discovery: Network scanning (T1046)
- Collection: Staging of large volumes of code repositories
- Exfiltration: HTTPS outbound to threat infrastructure (T1041)

Impacted Assets:
- ENG-JB01 (Jump Box, staging point)
- FIN-SQL-02 (Finance DB server, used for lateral movement)
- ENG-GIT-SRV (Code repository server, source of exfiltrated data)

Impacted Data:
- At least 12 private Git repositories accessed
- Two contained proprietary ML models and customer integration code
- No indication of PII or PCI data exposure

Root Cause:
The attacker appears to have compromised or reused valid credentials on ENG-JB01, which had elevated network permissions. The exact credential acquisition method is undetermined but likely due to reuse or prior phishing compromise. MFA was not enabled on the jump box account.

Remediation Actions:
- Immediate network isolation of all affected systems
- Reset of all privileged and domain service accounts
- Implementation of LAPS and forced password rotation for local accounts
- Blocking outbound traffic to 185.44.76.100 at the firewall
- Deployment of EDR response rules for persistence techniques
- Initiated rollout of MFA to all bastion and admin systems
- Engaged law enforcement and threat intelligence partners for IOCs

Lessons Learned:
- MFA gaps on high-privilege infrastructure allowed credential reuse
- Scheduled task creation alerts were not being monitored
- EDR rules for base64-encoded PowerShell were disabled in engineering OU
- Repository audit logging was insufficient to detect staging early

Recommendations:
- Mandatory MFA on all privileged and remote-access accounts
- Enable audit logging on all code and data staging systems
- Expand scheduled task alerting across all domains
- Conduct purple team simulation to test lateral movement detection

This report has been reviewed and approved by the incident response lead and executive security team. Further updates will be shared in postmortem review.

-- End of Report --
"""


In [ ]:
summary_exec = run_incident_summary_analysis(long_incident_doc, summary_type="executive")
print("🔎 Executive Summary:\n", summary_exec)

🔎 Executive Summary:
 **Summary:** On April 4th, attackers breached our network through an unsecured jump box (ENG-JB01), leading to an 18-hour intrusion where they stole sensitive source code from our repositories and tried to send it to a known APT29 server. They used stolen credentials, obfuscated PowerShell commands, and scheduled tasks for persistence. We've isolated the affected systems, blocked the exfiltration channel, and started fixing MFA and other security gaps. This highlights the need for stronger access controls and monitoring, especially around critical assets like code repositories.


In [ ]:
summary_tactical = run_incident_summary_analysis(long_incident_doc, summary_type="tactical")
print("🛠️ Tactical Findings:\n", summary_tactical)

🛠️ Tactical Findings:
 **Incident Summary: IR-2025-0173**

- **Initial Compromise:** Anomalous lateral movement detected from ENG-JB01 (April 4, 01:14 UTC) via possible Pass-the-Hash (T1550.002) leading to unauthorized access on FIN-SQL-02 using svc-backup-prod account.
- **Execution & Persistence:** PowerShell obfuscation (T1059.001) and scheduled task "WindowsHealth" (T1053.005) for persistence on FIN-SQL-02.
- **Discovery & Collection:** Network scanning (T1046) and SMB/WMI access to ENG-GIT-SRV, followed by exfiltration of 3.1GB of source code repositories.
- **Exfiltration:** Data staged and transferred over HTTPS to APT29-linked infrastructure (185.44.76.100) between 02:15–04:52 UTC.
- **Isolation & Forensics:** Systems isolated (ENG-JB01, FIN-SQL-02, ENG-GIT-SRV) after exfiltration detected; memory captures initiated.
- **Affected Assets:** ENG-JB01, FIN-SQL-02, ENG-GIT-SRV.
- **Data Impact:** Sensitive source code stolen, no PII or PCI exposed.
- **Root Cause:** Credential reus